# Abstractive:
generate new text that captures the most relevant information.


In [1]:
!pip install transformers datasets evaluate rouge_score

     ---------------------------------------- 7.0/7.0 MB 389.7 kB/s eta 0:00:00
     ------------------------------------ 474.6/474.6 kB 306.3 kB/s eta 0:00:00
     -------------------------------------- 81.4/81.4 kB 228.2 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 151.6/151.6 kB 335.1 kB/s eta 0:00:00
     ------------------------------------ 224.5/224.5 kB 456.8 kB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 509.0 kB/s eta 0:00:00
     -------------------------------------- 110.5/110.5 kB 1.1 MB/s eta 0:00:00
     ------------------------------------ 154.0/154.0 kB 542.4 kB/s eta 0:00:00
     -------------------------------------- 20.6/20.6 MB 826.1 kB/s eta 0:00:00
     -------------------------------------- 132.9/132.9 kB 1.1 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=867


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install --upgrade pip


     ---------------------------------------- 2.1/2.1 MB 205.1 kB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [6]:
# so we are using the billsum data set here
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

Found cached dataset billsum (C:/Users/vishal567795/.cache/huggingface/datasets/billsum/default/3.0.0/75cf1719d38d6553aa0e0714c393c74579b083ae6e164b2543684e3e92e0c4cc)


In [7]:
#Split the dataset into a train and test set with the train_test_split method:
billsum = billsum.train_test_split(test_size=0.2)

In [8]:
billsum["train"][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 1341.45 of the Health and Safety Code is amended to read:\n1341.45.\n(a) There is hereby created in the State Treasury the Managed Care Administrative Fines and Penalties Fund.\n(b) The fines and administrative penalties collected pursuant to this chapter, on and after September 30, 2008, shall be deposited into the Managed Care Administrative Fines and Penalties Fund.\n(c) The fines and administrative penalties deposited into the Managed Care Administrative Fines and Penalties Fund shall be transferred by the department, annually, as follows:\n(1) The first one million dollars ($1,000,000) shall be transferred to the Medically Underserved Account for Physicians within the Health Professions Education Fund and shall, upon appropriation by the Legislature, be used for the purposes of the Steven M. Thompson Physician Corps Loan Repayment Program, as specified in Article 5 (commencing with Sectio

There are two fields that you’ll want to use:

text: the text of the bill which’ll be the input to the model.
summary: a condensed version of text which’ll be the model target.

# Preprocessing

In [9]:
# The next step is to load a T5 tokenizer to process text and summary:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

c:\users\vishal567795\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vishal567795\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
